# 63-微服务架构设计

## 📚 用途说明

**学习目标**：
- 掌握微服务架构的设计原理和最佳实践
- 学会实现服务发现和通信机制
- 理解分布式系统的容错和监控
- 能够构建完整的微服务系统

**前置要求**：
- 已完成Web安全和前后端分离学习
- 熟练掌握FastAPI和异步编程
- 了解分布式系统基础概念
- 理解容器化和编排基础

**与LangChain关联**：
- 微服务架构支持LangChain的AI服务分布式部署
- 服务发现机制保障LangChain组件间通信
- 容错模式提升LangChain系统稳定性
- 为LangChain提供企业级微服务解决方案

---

## 🔢 知识点覆盖

### 7.8 微服务架构设计 [⭐⭐进阶]
**知识点说明**：微服务架构设计是现代分布式系统的核心模式，为LangChain的企业级部署和AI服务治理提供重要的技术支撑。

**学习要求**：
- 掌握微服务架构的设计原理和最佳实践
- 学会实现服务发现和通信机制
- 理解分布式系统的容错和监控
- 能够构建完整的微服务系统

**案例要求**：
- 实现完整的微服务架构系统
- 构建服务发现和通信机制
- 开发容错和监控模式
- 验证点：能独立设计和部署微服务系统

In [ ]:
print("🏗️ 微服务架构设计:")
print("=" * 50)

# 导入必要的库
import asyncio
import time
import datetime
import uuid
import json
import random
import re
import hashlib
import logging
from typing import List, Dict, Any, Optional, Union, Type, Callable
from dataclasses import dataclass, asdict, field
from enum import Enum
from contextlib import asynccontextmanager
from pathlib import Path
import threading
from concurrent.futures import ThreadPoolExecutor

# FastAPI相关
from fastapi import FastAPI, HTTPException, Depends, status, Request, Response, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from pydantic import BaseModel, EmailStr, Field, validator

# HTTP客户端
import aiohttp
import requests

# Redis用于消息队列和缓存
try:
    import redis
    import aioredis
except ImportError:
    print("警告: Redis库未安装，某些功能将不可用")
    redis = None
    aioredis = None

print(f"✅ Python版本: {__import__('sys').version}")
print(f"✅ 当前时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 1. 微服务核心概念
print(f"📝 1. 微服务核心概念:")

# 1.1 服务定义和配置
print(f"\n   🏛️ 1.1 服务定义和配置:")

@dataclass
class ServiceConfig:
    """服务配置"""
    name: str
    version: str
    host: str = "localhost"
    port: int = 8000
    health_check_path: str = "/health"
    metrics_path: str = "/metrics"
    dependencies: List[str] = field(default_factory=list)
    tags: List[str] = field(default_factory=list)
    environment: str = "development"
    
    @property
    def base_url(self) -> str:
        return f"http://{self.host}:{self.port}"
    
    @property
    def service_id(self) -> str:
        return f"{self.name}:{self.version}"

@dataclass
class ServiceInstance:
    """服务实例"""
    config: ServiceConfig
    instance_id: str = field(default_factory=lambda: str(uuid.uuid4()))
    status: str = "starting"  # starting, running, stopping, stopped
    registered_at: datetime.datetime = field(default_factory=datetime.datetime.utcnow)
    last_heartbeat: datetime.datetime = field(default_factory=datetime.datetime.utcnow)
    metadata: Dict[str, Any] = field(default_factory=dict)
    
    def is_healthy(self, timeout: int = 30) -> bool:
        """检查服务是否健康"""
        if self.status != "running":
            return False
        
        time_since_heartbeat = (datetime.datetime.utcnow() - self.last_heartbeat).total_seconds()
        return time_since_heartbeat <= timeout

class ServiceRegistry:
    """服务注册中心"""
    
    def __init__(self):
        self.services: Dict[str, List[ServiceInstance]] = {}
        self.lock = threading.Lock()
    
    def register(self, instance: ServiceInstance) -> bool:
        """注册服务实例"""
        with self.lock:
            service_id = instance.config.service_id
            if service_id not in self.services:
                self.services[service_id] = []
            
            # 检查是否已存在相同实例
            for existing in self.services[service_id]:
                if existing.instance_id == instance.instance_id:
                    # 更新现有实例
                    existing.last_heartbeat = datetime.datetime.utcnow()
                    existing.status = instance.status
                    return True
            
            # 添加新实例
            self.services[service_id].append(instance)
            instance.status = "running"
            return True
    
    def deregister(self, service_id: str, instance_id: str) -> bool:
        """注销服务实例"""
        with self.lock:
            if service_id not in self.services:
                return False
            
            for i, instance in enumerate(self.services[service_id]):
                if instance.instance_id == instance_id:
                    instance.status = "stopped"
                    self.services[service_id].pop(i)
                    return True
            
            return False
    
    def discover(self, service_id: str, healthy_only: bool = True) -> List[ServiceInstance]:
        """发现服务实例"""
        with self.lock:
            instances = self.services.get(service_id, [])
            
            if healthy_only:
                instances = [inst for inst in instances if inst.is_healthy()]
            
            return instances.copy()
    
    def heartbeat(self, service_id: str, instance_id: str) -> bool:
        """服务心跳"""
        with self.lock:
            if service_id not in self.services:
                return False
            
            for instance in self.services[service_id]:
                if instance.instance_id == instance_id:
                    instance.last_heartbeat = datetime.datetime.utcnow()
                    return True
            
            return False
    
    def get_all_services(self) -> Dict[str, List[ServiceInstance]]:
        """获取所有服务"""
        with self.lock:
            return {k: v.copy() for k, v in self.services.items()}
    
    def cleanup_stale_instances(self, timeout: int = 60) -> int:
        """清理过期实例"""
        cleaned_count = 0
        with self.lock:
            for service_id, instances in list(self.services.items()):
                healthy_instances = []
                for instance in instances:
                    if instance.is_healthy(timeout):
                        healthy_instances.append(instance)
                    else:
                        cleaned_count += 1
                
                if healthy_instances:
                    self.services[service_id] = healthy_instances
                else:
                    del self.services[service_id]
        
        return cleaned_count

# 1.2 负载均衡器
print(f"\n   ⚖️ 1.2 负载均衡器:")

class LoadBalancerStrategy(Enum):
    """负载均衡策略"""
    ROUND_ROBIN = "round_robin"
    RANDOM = "random"
    LEAST_CONNECTIONS = "least_connections"
    WEIGHTED_ROUND_ROBIN = "weighted_round_robin"

class LoadBalancer:
    """负载均衡器"""
    
    def __init__(self, strategy: LoadBalancerStrategy = LoadBalancerStrategy.ROUND_ROBIN):
        self.strategy = strategy
        self.round_robin_counters = {}
        self.connection_counts = {}
    
    def select_instance(self, instances: List[ServiceInstance]) -> Optional[ServiceInstance]:
        """选择服务实例"""
        if not instances:
            return None
        
        healthy_instances = [inst for inst in instances if inst.is_healthy()]
        if not healthy_instances:
            return None
        
        if self.strategy == LoadBalancerStrategy.ROUND_ROBIN:
            return self._round_robin_select(healthy_instances)
        elif self.strategy == LoadBalancerStrategy.RANDOM:
            return self._random_select(healthy_instances)
        elif self.strategy == LoadBalancerStrategy.LEAST_CONNECTIONS:
            return self._least_connections_select(healthy_instances)
        else:
            return healthy_instances[0]
    
    def _round_robin_select(self, instances: List[ServiceInstance]) -> ServiceInstance:
        """轮询选择"""
        service_id = instances[0].config.service_id
        
        if service_id not in self.round_robin_counters:
            self.round_robin_counters[service_id] = 0
        
        index = self.round_robin_counters[service_id] % len(instances)
        self.round_robin_counters[service_id] += 1
        
        return instances[index]
    
    def _random_select(self, instances: List[ServiceInstance]) -> ServiceInstance:
        """随机选择"""
        return random.choice(instances)
    
    def _least_connections_select(self, instances: List[ServiceInstance]) -> ServiceInstance:
        """最少连接选择"""
        min_connections = float('inf')
        selected_instance = None
        
        for instance in instances:
            connections = self.connection_counts.get(instance.instance_id, 0)
            if connections < min_connections:
                min_connections = connections
                selected_instance = instance
        
        return selected_instance
    
    def increment_connections(self, instance_id: str):
        """增加连接计数"""
        self.connection_counts[instance_id] = self.connection_counts.get(instance_id, 0) + 1
    
    def decrement_connections(self, instance_id: str):
        """减少连接计数"""
        if instance_id in self.connection_counts:
            self.connection_counts[instance_id] = max(0, self.connection_counts[instance_id] - 1)

print(f"   ✅ 微服务核心概念完成")
print(f"      - ServiceConfig: 服务配置")
print(f"      - ServiceInstance: 服务实例")
print(f"      - ServiceRegistry: 服务注册中心")
print(f"      - LoadBalancer: 负载均衡器")

print(f"\n✅ 微服务核心概念完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握微服务架构的设计原理和最佳实践")
print(f"   ✓ 学会实现服务发现和通信机制")
print(f"   ✓ 理解分布式系统的容错和监控")
print(f"   ✓ 能够构建完整的微服务系统")

### 服务间通信与API网关 [⭐⭐进阶]
**知识点说明**：服务间通信与API网关是微服务架构的核心组件，为LangChain的分布式AI服务提供统一的通信入口和服务协调机制。

**学习要求**：
- 掌握服务间通信模式和协议
- 学会实现API网关和路由
- 理解异步消息传递机制
- 能够设计高效的服务通信系统

**案例要求**：
- 实现完整的服务通信系统
- 构建API网关和路由机制
- 开发异步消息传递
- 验证点：能独立设计服务通信架构

In [ ]:
print("\n🔗 服务间通信与API网关:")
print("=" * 50)

# 2. 服务间通信与API网关
print(f"📝 2. 服务间通信与API网关:")

# 2.1 服务通信客户端
print(f"\n   📡 2.1 服务通信客户端:")

@dataclass
class ServiceCall:
    """服务调用记录"""
    service_id: str
    method: str
    path: str
    headers: Dict[str, str] = field(default_factory=dict)
    params: Dict[str, Any] = field(default_factory=dict)
    data: Any = None
    timeout: float = 30.0
    retry_count: int = 3
    
    @property
    def call_id(self) -> str:
        return str(uuid.uuid4())

class ServiceCommunicator:
    """服务通信器"""
    
    def __init__(self, registry: ServiceRegistry, load_balancer: LoadBalancer):
        self.registry = registry
        self.load_balancer = load_balancer
        self.circuit_breakers = {}
        self.call_history = []
    
    async def call_service(self, service_call: ServiceCall) -> Dict[str, Any]:
        """调用服务"""
        start_time = time.time()
        
        try:
            # 发现服务实例
            instances = self.registry.discover(service_call.service_id)
            instance = self.load_balancer.select_instance(instances)
            
            if not instance:
                raise Exception(f"服务 {service_call.service_id} 不可用")
            
            # 检查熔断器
            if self._is_circuit_open(service_call.service_id):
                raise Exception(f"服务 {service_call.service_id} 熔断器已开启")
            
            # 发送请求
            url = f"{instance.config.base_url}{service_call.path}"
            
            self.load_balancer.increment_connections(instance.instance_id)
            
            try:
                async with aiohttp.ClientSession() as session:
                    async with session.request(
                        method=service_call.method,
                        url=url,
                        headers=service_call.headers,
                        params=service_call.params,
                        json=service_call.data if service_call.data else None,
                        timeout=aiohttp.ClientTimeout(total=service_call.timeout)
                    ) as response:
                        result = await response.json()
                        
                        if response.status >= 400:
                            raise Exception(f"服务调用失败: {response.status}")
                        
                        # 记录成功调用
                        self._record_call(service_call, instance, True, time.time() - start_time)
                        self._reset_circuit_breaker(service_call.service_id)
                        
                        return result
            
            except Exception as e:
                # 记录失败调用
                self._record_call(service_call, instance, False, time.time() - start_time)
                self._handle_circuit_breaker(service_call.service_id)
                raise e
            
            finally:
                self.load_balancer.decrement_connections(instance.instance_id)
        
        except Exception as e:
            # 重试逻辑
            if service_call.retry_count > 0:
                service_call.retry_count -= 1
                await asyncio.sleep(1)  # 等待1秒后重试
                return await self.call_service(service_call)
            else:
                raise e
    
    def _is_circuit_open(self, service_id: str) -> bool:
        """检查熔断器是否开启"""
        if service_id not in self.circuit_breakers:
            return False
        
        breaker = self.circuit_breakers[service_id]
        
        if breaker["state"] == "open":
            # 检查是否可以进入半开状态
            if time.time() - breaker["last_failure_time"] > breaker["timeout"]:
                breaker["state"] = "half_open"
                return False
            return True
        
        return False
    
    def _handle_circuit_breaker(self, service_id: str):
        """处理熔断器"""
        if service_id not in self.circuit_breakers:
            self.circuit_breakers[service_id] = {
                "state": "closed",
                "failure_count": 0,
                "failure_threshold": 5,
                "timeout": 60,
                "last_failure_time": 0
            }
        
        breaker = self.circuit_breakers[service_id]
        breaker["failure_count"] += 1
        breaker["last_failure_time"] = time.time()
        
        if breaker["failure_count"] >= breaker["failure_threshold"]:
            breaker["state"] = "open"
    
    def _reset_circuit_breaker(self, service_id: str):
        """重置熔断器"""
        if service_id in self.circuit_breakers:
            breaker = self.circuit_breakers[service_id]
            if breaker["state"] == "half_open":
                breaker["state"] = "closed"
                breaker["failure_count"] = 0
    
    def _record_call(self, service_call: ServiceCall, instance: ServiceInstance, 
                    success: bool, duration: float):
        """记录调用历史"""
        self.call_history.append({
            "call_id": service_call.call_id,
            "service_id": service_call.service_id,
            "instance_id": instance.instance_id,
            "method": service_call.method,
            "path": service_call.path,
            "success": success,
            "duration": duration,
            "timestamp": datetime.datetime.utcnow()
        })
        
        # 保持历史记录在合理范围内
        if len(self.call_history) > 1000:
            self.call_history = self.call_history[-500:]
    
    def get_call_statistics(self, service_id: str = None) -> Dict[str, Any]:
        """获取调用统计"""
        calls = self.call_history
        
        if service_id:
            calls = [call for call in calls if call["service_id"] == service_id]
        
        if not calls:
            return {"total_calls": 0}
        
        total_calls = len(calls)
        successful_calls = sum(1 for call in calls if call["success"])
        failed_calls = total_calls - successful_calls
        
        avg_duration = sum(call["duration"] for call in calls) / total_calls
        
        return {
            "total_calls": total_calls,
            "successful_calls": successful_calls,
            "failed_calls": failed_calls,
            "success_rate": successful_calls / total_calls * 100,
            "average_duration": avg_duration,
            "circuit_breaker_status": self.circuit_breakers.get(service_id, {"state": "closed"})
        }

# 2.2 API网关
print(f"\n   🚪 2.2 API网关:")

@dataclass
class RouteConfig:
    """路由配置"""
    path: str
    service_id: str
    methods: List[str] = field(default_factory=lambda: ["GET"])
    rewrite_path: Optional[str] = None
    timeout: float = 30.0
    retry_count: int = 3
    rate_limit: Optional[int] = None  # 每分钟请求限制
    auth_required: bool = False
    
class APIGateway:
    """API网关"""
    
    def __init__(self, registry: ServiceRegistry, load_balancer: LoadBalancer):
        self.registry = registry
        self.load_balancer = load_balancer
        self.communicator = ServiceCommunicator(registry, load_balancer)
        self.routes: Dict[str, RouteConfig] = {}
        self.rate_limiters = {}
        self.request_logs = []
    
    def add_route(self, route_config: RouteConfig):
        """添加路由"""
        self.routes[route_config.path] = route_config
    
    async def handle_request(self, request: Request) -> Response:
        """处理请求"""
        start_time = time.time()
        
        try:
            # 查找匹配的路由
            route_config = self._find_route(request.url.path, request.method)
            
            if not route_config:
                return JSONResponse(
                    status_code=404,
                    content={"error": "路由未找到"}
                )
            
            # 速率限制检查
            client_ip = request.client.host
            if not self._check_rate_limit(client_ip, route_config):
                return JSONResponse(
                    status_code=429,
                    content={"error": "请求过于频繁"}
                )
            
            # 认证检查
            if route_config.auth_required:
                auth_result = await self._check_authentication(request)
                if not auth_result["valid"]:
                    return JSONResponse(
                        status_code=401,
                        content={"error": "认证失败"}
                    )
            
            # 准备服务调用
            target_path = route_config.rewrite_path or request.url.path
            
            service_call = ServiceCall(
                service_id=route_config.service_id,
                method=request.method,
                path=target_path,
                headers=dict(request.headers),
                params=dict(request.query_params),
                timeout=route_config.timeout,
                retry_count=route_config.retry_count
            )
            
            # 处理请求体
            if request.method in ["POST", "PUT", "PATCH"]:
                try:
                    service_call.data = await request.json()
                except Exception:
                    service_call.data = await request.form()
            
            # 调用后端服务
            result = await self.communicator.call_service(service_call)
            
            # 记录请求日志
            self._log_request(request, route_config, True, time.time() - start_time)
            
            return JSONResponse(content=result)
        
        except Exception as e:
            # 记录错误日志
            self._log_request(request, route_config if 'route_config' in locals() else None, 
                            False, time.time() - start_time, str(e))
            
            return JSONResponse(
                status_code=500,
                content={"error": "网关内部错误"}
            )
    
    def _find_route(self, path: str, method: str) -> Optional[RouteConfig]:
        """查找匹配的路由"""
        for route_path, route_config in self.routes.items():
            if self._path_matches(path, route_path) and method in route_config.methods:
                return route_config
        return None
    
    def _path_matches(self, request_path: str, route_path: str) -> bool:
        """检查路径是否匹配"""
        # 简单的路径匹配，支持通配符
        if route_path.endswith("*"):
            prefix = route_path[:-1]
            return request_path.startswith(prefix)
        else:
            return request_path == route_path
    
    def _check_rate_limit(self, client_ip: str, route_config: RouteConfig) -> bool:
        """检查速率限制"""
        if not route_config.rate_limit:
            return True
        
        key = f"{client_ip}:{route_config.path}"
        current_time = time.time()
        
        if key not in self.rate_limiters:
            self.rate_limiters[key] = []
        
        # 清理过期的请求记录
        self.rate_limiters[key] = [
            req_time for req_time in self.rate_limiters[key]
            if current_time - req_time < 60  # 1分钟内
        ]
        
        # 检查是否超过限制
        if len(self.rate_limiters[key]) >= route_config.rate_limit:
            return False
        
        # 记录当前请求
        self.rate_limiters[key].append(current_time)
        return True
    
    async def _check_authentication(self, request: Request) -> Dict[str, Any]:
        """检查认证"""
        # 简单的JWT认证检查
        auth_header = request.headers.get("Authorization")
        
        if not auth_header or not auth_header.startswith("Bearer "):
            return {"valid": False, "error": "缺少认证令牌"}
        
        token = auth_header[7:]  # 移除 "Bearer " 前缀
        
        # 这里应该验证JWT令牌，简化处理
        if token and len(token) > 10:
            return {"valid": True}
        
        return {"valid": False, "error": "无效的认证令牌"}
    
    def _log_request(self, request: Request, route_config: Optional[RouteConfig], 
                    success: bool, duration: float, error: str = None):
        """记录请求日志"""
        log_entry = {
            "timestamp": datetime.datetime.utcnow(),
            "method": request.method,
            "path": request.url.path,
            "client_ip": request.client.host,
            "user_agent": request.headers.get("User-Agent"),
            "service_id": route_config.service_id if route_config else None,
            "success": success,
            "duration": duration,
            "error": error
        }
        
        self.request_logs.append(log_entry)
        
        # 保持日志在合理范围内
        if len(self.request_logs) > 1000:
            self.request_logs = self.request_logs[-500:]
    
    def get_gateway_statistics(self) -> Dict[str, Any]:
        """获取网关统计信息"""
        if not self.request_logs:
            return {"total_requests": 0}
        
        total_requests = len(self.request_logs)
        successful_requests = sum(1 for log in self.request_logs if log["success"])
        failed_requests = total_requests - successful_requests
        
        avg_duration = sum(log["duration"] for log in self.request_logs) / total_requests
        
        # 按服务统计
        service_stats = {}
        for log in self.request_logs:
            service_id = log["service_id"] or "unknown"
            if service_id not in service_stats:
                service_stats[service_id] = {"requests": 0, "successes": 0}
            
            service_stats[service_id]["requests"] += 1
            if log["success"]:
                service_stats[service_id]["successes"] += 1
        
        return {
            "total_requests": total_requests,
            "successful_requests": successful_requests,
            "failed_requests": failed_requests,
            "success_rate": successful_requests / total_requests * 100,
            "average_duration": avg_duration,
            "active_routes": len(self.routes),
            "circuit_breakers": len(self.communicator.circuit_breakers),
            "service_statistics": service_stats
        }

print(f"   ✅ 服务间通信与API网关完成")
print(f"      - ServiceCall: 服务调用记录")
print(f"      - ServiceCommunicator: 服务通信器")
print(f"      - RouteConfig: 路由配置")
print(f"      - APIGateway: API网关")

print(f"\n✅ 服务间通信与API网关完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握服务间通信模式和协议")
print(f"   ✓ 学会实现API网关和路由")
print(f"   ✓ 理解异步消息传递机制")
print(f"   ✓ 能够设计高效的服务通信系统")

### 分布式数据与容错模式 [⭐⭐进阶]
**知识点说明**：分布式数据与容错模式是微服务架构的高级特性，为LangChain的AI服务提供数据一致性保障和系统稳定性。

**学习要求**：
- 掌握分布式数据管理模式
- 学会实现容错和恢复机制
- 理解分布式事务处理
- 能够设计高可用的微服务系统

**案例要求**：
- 实现分布式数据管理系统
- 构建容错和恢复机制
- 开发分布式事务处理
- 验证点：能独立设计高可用微服务系统

In [ ]:
print("\n🔄 分布式数据与容错模式:")
print("=" * 50)

# 3. 分布式数据与容错模式
print(f"📝 3. 分布式数据与容错模式:")

# 3.1 分布式事务管理
print(f"\n   💾 3.1 分布式事务管理:")

@dataclass
class TransactionStep:
    """事务步骤"""
    step_id: str
    service_id: str
    action: str
    data: Dict[str, Any]
    compensate_action: Optional[str] = None
    compensate_data: Optional[Dict[str, Any]] = None
    status: str = "pending"  # pending, completed, failed, compensated
    retry_count: int = 0
    max_retries: int = 3

@dataclass
class SagaTransaction:
    """Saga事务"""
    transaction_id: str
    steps: List[TransactionStep]
    current_step: int = 0
    status: str = "pending"  # pending, running, completed, failed, compensating
    created_at: datetime.datetime = field(default_factory=datetime.datetime.utcnow)
    updated_at: datetime.datetime = field(default_factory=datetime.datetime.utcnow)
    
    def get_current_step(self) -> Optional[TransactionStep]:
        """获取当前步骤"""
        if 0 <= self.current_step < len(self.steps):
            return self.steps[self.current_step]
        return None
    
    def get_completed_steps(self) -> List[TransactionStep]:
        """获取已完成的步骤"""
        return [step for step in self.steps if step.status == "completed"]
    
    def get_failed_steps(self) -> List[TransactionStep]:
        """获取失败的步骤"""
        return [step for step in self.steps if step.status == "failed"]

class SagaManager:
    """Saga管理器"""
    
    def __init__(self, communicator: ServiceCommunicator):
        self.communicator = communicator
        self.active_transactions: Dict[str, SagaTransaction] = {}
        self.transaction_history = []
    
    async def execute_saga(self, saga: SagaTransaction) -> Dict[str, Any]:
        """执行Saga事务"""
        self.active_transactions[saga.transaction_id] = saga
        saga.status = "running"
        
        try:
            # 执行所有步骤
            for i, step in enumerate(saga.steps):
                saga.current_step = i
                
                success = await self._execute_step(step)
                
                if not success:
                    # 步骤失败，开始补偿
                    saga.status = "compensating"
                    await self._compensate_transaction(saga)
                    saga.status = "failed"
                    break
            
            if saga.status == "running":
                saga.status = "completed"
            
            # 记录事务历史
            self._record_transaction(saga)
            
            return {
                "transaction_id": saga.transaction_id,
                "status": saga.status,
                "completed_steps": len(saga.get_completed_steps()),
                "total_steps": len(saga.steps)
            }
        
        except Exception as e:
            saga.status = "failed"
            self._record_transaction(saga)
            raise e
        
        finally:
            # 清理活跃事务
            if saga.transaction_id in self.active_transactions:
                del self.active_transactions[saga.transaction_id]
    
    async def _execute_step(self, step: TransactionStep) -> bool:
        """执行单个步骤"""
        for attempt in range(step.max_retries + 1):
            try:
                service_call = ServiceCall(
                    service_id=step.service_id,
                    method="POST",
                    path=f"/actions/{step.action}",
                    data=step.data,
                    timeout=30.0,
                    retry_count=0
                )
                
                result = await self.communicator.call_service(service_call)
                
                if result.get("success"):
                    step.status = "completed"
                    return True
                else:
                    step.status = "failed"
                    return False
            
            except Exception as e:
                step.retry_count += 1
                if attempt < step.max_retries:
                    await asyncio.sleep(2 ** attempt)  # 指数退避
                    continue
                else:
                    step.status = "failed"
                    return False
        
        return False
    
    async def _compensate_transaction(self, saga: SagaTransaction):
        """补偿事务（回滚）"""
        # 按相反顺序补偿已完成的步骤
        completed_steps = saga.get_completed_steps()
        
        for step in reversed(completed_steps):
            if step.compensate_action:
                try:
                    service_call = ServiceCall(
                        service_id=step.service_id,
                        method="POST",
                        path=f"/actions/{step.compensate_action}",
                        data=step.compensate_data or {},
                        timeout=30.0,
                        retry_count=1
                    )
                    
                    result = await self.communicator.call_service(service_call)
                    
                    if result.get("success"):
                        step.status = "compensated"
                    else:
                        # 补偿失败，记录但继续
                        step.status = "compensation_failed"
                
                except Exception:
                    step.status = "compensation_failed"
    
    def _record_transaction(self, saga: SagaTransaction):
        """记录事务历史"""
        self.transaction_history.append({
            "transaction_id": saga.transaction_id,
            "status": saga.status,
            "total_steps": len(saga.steps),
            "completed_steps": len(saga.get_completed_steps()),
            "failed_steps": len(saga.get_failed_steps()),
            "duration": (saga.updated_at - saga.created_at).total_seconds(),
            "created_at": saga.created_at
        })
        
        # 保持历史记录在合理范围内
        if len(self.transaction_history) > 1000:
            self.transaction_history = self.transaction_history[-500:]
    
    def get_transaction_statistics(self) -> Dict[str, Any]:
        """获取事务统计"""
        if not self.transaction_history:
            return {"total_transactions": 0}
        
        total_transactions = len(self.transaction_history)
        completed_transactions = sum(1 for tx in self.transaction_history if tx["status"] == "completed")
        failed_transactions = sum(1 for tx in self.transaction_history if tx["status"] == "failed")
        
        avg_duration = sum(tx["duration"] for tx in self.transaction_history) / total_transactions
        
        return {
            "total_transactions": total_transactions,
            "completed_transactions": completed_transactions,
            "failed_transactions": failed_transactions,
            "success_rate": completed_transactions / total_transactions * 100,
            "average_duration": avg_duration,
            "active_transactions": len(self.active_transactions)
        }

# 3.2 容错和重试机制
print(f"\n   🛡️ 3.2 容错和重试机制:")

@dataclass
class RetryPolicy:
    """重试策略"""
    max_attempts: int = 3
    initial_delay: float = 1.0
    max_delay: float = 60.0
    multiplier: float = 2.0
    jitter: bool = True
    
    def get_delay(self, attempt: int) -> float:
        """获取重试延迟"""
        delay = min(self.initial_delay * (self.multiplier ** attempt), self.max_delay)
        
        if self.jitter:
            delay *= (0.5 + random.random() * 0.5)
        
        return delay

class ResilienceManager:
    """弹性管理器"""
    
    def __init__(self):
        self.retry_policies: Dict[str, RetryPolicy] = {}
        self.bulkheads: Dict[str, Dict[str, Any]] = {}
        self.timeouts: Dict[str, float] = {}
        self.fallback_handlers: Dict[str, Callable] = {}
    
    def set_retry_policy(self, service_id: str, policy: RetryPolicy):
        """设置重试策略"""
        self.retry_policies[service_id] = policy
    
    def configure_bulkhead(self, service_id: str, max_concurrent: int, max_queue: int):
        """配置舱壁隔离"""
        self.bulkheads[service_id] = {
            "max_concurrent": max_concurrent,
            "max_queue": max_queue,
            "current_concurrent": 0,
            "queue": asyncio.Queue(maxsize=max_queue)
        }
    
    def set_timeout(self, service_id: str, timeout: float):
        """设置超时"""
        self.timeouts[service_id] = timeout
    
    def set_fallback(self, service_id: str, handler: Callable):
        """设置降级处理"""
        self.fallback_handlers[service_id] = handler
    
    async def execute_with_resilience(self, service_id: str, 
                                      operation: Callable, *args, **kwargs) -> Any:
        """使用弹性模式执行操作"""
        retry_policy = self.retry_policies.get(service_id, RetryPolicy())
        timeout = self.timeouts.get(service_id, 30.0)
        
        # 舱壁隔离检查
        if service_id in self.bulkheads:
            bulkhead = self.bulkheads[service_id]
            if bulkhead["current_concurrent"] >= bulkhead["max_concurrent"]:
                if bulkhead["queue"].full():
                    # 队列已满，执行降级
                    if service_id in self.fallback_handlers:
                        return await self.fallback_handlers[service_id](*args, **kwargs)
                    else:
                        raise Exception("服务过载，队列已满")
                
                # 加入队列
                await bulkhead["queue"].put(None)
        
        # 执行操作（带重试）
        last_exception = None
        
        for attempt in range(retry_policy.max_attempts):
            try:
                # 增加并发计数
                if service_id in self.bulkheads:
                    self.bulkheads[service_id]["current_concurrent"] += 1
                
                # 执行操作（带超时）
                result = await asyncio.wait_for(operation(*args, **kwargs), timeout=timeout)
                
                return result
            
            except Exception as e:
                last_exception = e
                
                if attempt < retry_policy.max_attempts - 1:
                    delay = retry_policy.get_delay(attempt)
                    await asyncio.sleep(delay)
                    continue
                else:
                    # 最后一次尝试失败，执行降级
                    if service_id in self.fallback_handlers:
                        return await self.fallback_handlers[service_id](*args, **kwargs)
                    else:
                        raise e
            
            finally:
                # 减少并发计数
                if service_id in self.bulkheads:
                    self.bulkheads[service_id]["current_concurrent"] -= 1
                    
                    # 从队列中移除
                    if not self.bulkheads[service_id]["queue"].empty():
                        await self.bulkheads[service_id]["queue"].get()
        
        raise last_exception

# 3.3 分布式监控
print(f"\n   📊 3.3 分布式监控:")

@dataclass
class MetricPoint:
    """指标点"""
    name: str
    value: float
    tags: Dict[str, str] = field(default_factory=dict)
    timestamp: datetime.datetime = field(default_factory=datetime.datetime.utcnow)

class DistributedMonitor:
    """分布式监控器"""
    
    def __init__(self):
        self.metrics: List[MetricPoint] = []
        self.alerts: List[Dict[str, Any]] = []
        self.alert_rules: List[Dict[str, Any]] = []
    
    def record_metric(self, name: str, value: float, tags: Dict[str, str] = None):
        """记录指标"""
        metric = MetricPoint(
            name=name,
            value=value,
            tags=tags or {}
        )
        
        self.metrics.append(metric)
        
        # 保持指标在合理范围内
        if len(self.metrics) > 10000:
            self.metrics = self.metrics[-5000:]
        
        # 检查告警规则
        self._check_alerts(metric)
    
    def add_alert_rule(self, name: str, metric_name: str, 
                      condition: str, threshold: float, 
                      duration: int = 300):
        """添加告警规则"""
        self.alert_rules.append({
            "name": name,
            "metric_name": metric_name,
            "condition": condition,  # >, <, >=, <=, ==
            "threshold": threshold,
            "duration": duration,
            "triggered_at": None
        })
    
    def _check_alerts(self, metric: MetricPoint):
        """检查告警"""
        for rule in self.alert_rules:
            if rule["metric_name"] != metric.name:
                continue
            
            condition_met = self._evaluate_condition(
                metric.value, rule["condition"], rule["threshold"]
            )
            
            if condition_met:
                if rule["triggered_at"] is None:
                    rule["triggered_at"] = time.time()
                elif time.time() - rule["triggered_at"] >= rule["duration"]:
                    # 触发告警
                    self._trigger_alert(rule, metric)
                    rule["triggered_at"] = None  # 重置
            else:
                rule["triggered_at"] = None
    
    def _evaluate_condition(self, value: float, condition: str, threshold: float) -> bool:
        """评估条件"""
        if condition == ">":
            return value > threshold
        elif condition == "<":
            return value < threshold
        elif condition == ">=":
            return value >= threshold
        elif condition == "<=":
            return value <= threshold
        elif condition == "==":
            return value == threshold
        return False
    
    def _trigger_alert(self, rule: Dict[str, Any], metric: MetricPoint):
        """触发告警"""
        alert = {
            "id": str(uuid.uuid4()),
            "rule_name": rule["name"],
            "metric_name": metric.name,
            "metric_value": metric.value,
            "threshold": rule["threshold"],
            "condition": rule["condition"],
            "timestamp": datetime.datetime.utcnow(),
            "tags": metric.tags
        }
        
        self.alerts.append(alert)
        
        # 保持告警在合理范围内
        if len(self.alerts) > 1000:
            self.alerts = self.alerts[-500:]
        
        print(f"🚨 告警触发: {alert['rule_name']} - {alert['metric_name']} = {alert['metric_value']}")
    
    def get_metrics_summary(self, metric_name: str = None, 
                           time_range: int = 3600) -> Dict[str, Any]:
        """获取指标摘要"""
        cutoff_time = datetime.datetime.utcnow() - datetime.timedelta(seconds=time_range)
        
        filtered_metrics = [
            metric for metric in self.metrics
            if metric.timestamp >= cutoff_time and (not metric_name or metric.name == metric_name)
        ]
        
        if not filtered_metrics:
            return {"count": 0}
        
        values = [metric.value for metric in filtered_metrics]
        
        return {
            "count": len(values),
            "min": min(values),
            "max": max(values),
            "avg": sum(values) / len(values),
            "latest": filtered_metrics[-1].value,
            "metric_name": metric_name or "all"
        }
    
    def get_active_alerts(self) -> List[Dict[str, Any]]:
        """获取活跃告警"""
        # 返回最近1小时的告警
        cutoff_time = datetime.datetime.utcnow() - datetime.timedelta(hours=1)
        
        return [
            alert for alert in self.alerts
            if alert["timestamp"] >= cutoff_time
        ]

print(f"   ✅ 分布式数据与容错模式完成")
print(f"      - SagaTransaction: Saga事务")
print(f"      - SagaManager: Saga管理器")
print(f"      - ResilienceManager: 弹性管理器")
print(f"      - DistributedMonitor: 分布式监控器")

print(f"\n✅ 分布式数据与容错模式完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握分布式数据管理模式")
print(f"   ✓ 学会实现容错和恢复机制")
print(f"   ✓ 理解分布式事务处理")
print(f"   ✓ 能够设计高可用的微服务系统")

## 📋 学习总结

### 知识清单达成情况

#### ✅ 已完成知识点

**7.8 微服务架构设计 [⭐⭐进阶]**
- ✅ 掌握微服务架构的设计原理和最佳实践
- ✅ 学会实现服务发现和通信机制
- ✅ 理解分布式系统的容错和监控
- ✅ 能够构建完整的微服务系统

#### 🎯 核心技能掌握

**微服务核心概念**
- ✅ 服务注册与发现机制
- ✅ 负载均衡策略实现
- ✅ 服务实例生命周期管理
- ✅ 健康检查和心跳机制

**服务通信与网关**
- ✅ 服务间通信模式设计
- ✅ API网关路由和代理
- ✅ 熔断器模式实现
- ✅ 速率限制和认证机制

**分布式数据与容错**
- ✅ Saga分布式事务管理
- ✅ 重试和舱壁隔离模式
- ✅ 降级和超时处理
- ✅ 分布式监控和告警

#### 🚀 实践应用能力

**架构设计**
- ✅ 完整的微服务架构设计
- ✅ 服务治理和协调机制
- ✅ 分布式系统可靠性保障
- ✅ 性能监控和故障诊断

**生产实践**
- ✅ 企业级微服务部署方案
- ✅ 高可用和容错设计
- ✅ 分布式事务一致性保障
- ✅ 实时监控和运维管理

#### 📊 与LangChain的关联

**AI服务分布式部署**
- ✅ 支持LangChain的AI服务微服务化
- ✅ 为LangChain提供分布式通信机制
- ✅ 保障LangChain服务的高可用性
- ✅ 实现LangChain组件的协调管理

**企业级解决方案**
- ✅ 满足LangChain企业级部署需求
- ✅ 提供AI服务的可扩展架构
- ✅ 保障LangChain系统的稳定运行
- ✅ 支持大规模AI应用场景

---

## 🎯 学习成果

通过本节学习，你已经掌握了：

1. **微服务架构设计** - 理解微服务原理，掌握服务发现、负载均衡和通信机制
2. **API网关实现** - 构建完整的API网关系统，掌握路由、认证和限流功能
3. **分布式事务管理** - 实现Saga模式，掌握分布式数据一致性保障机制
4. **容错和监控** - 设计弹性系统，掌握熔断、重试、降级和监控告警

这些技能为后续的Web应用部署、项目工程等高级主题奠定了坚实基础，也为LangChain的企业级微服务架构和AI服务治理提供了重要的技术支撑。